#### **Homework 7**

In [130]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

sns.set_style('whitegrid')

In [131]:
!git clone "https://github.com/HarshvardhanSingh-13/Datasets"

fatal: destination path 'Datasets' already exists and is not an empty directory.


In [132]:
df = pd.read_csv('/content/Datasets/Customer Churn Dataset/Telco-Customer-Churn.csv')

#### First we analize the dataset

In [133]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [134]:
df['TotalCharges'].value_counts()

,count
TotalCharges,
,11
20.2,11
19.75,9
20.05,8
19.9,8
...,...
130.15,1
3211.9,1
7843.55,1


In [135]:
df.shape

(7043, 21)

In [136]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [137]:
df.describe()

,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


We see that TotalCharges it's not numeric,
so we convert to numeric value.

In [138]:
df['TotalCharges']=pd.to_numeric(df['TotalCharges'],errors='coerce')
df

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.50,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.90,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.60,Yes


In [139]:
df.isnull().sum()

,0
customerID,0
gender,0
SeniorCitizen,0
Partner,0
Dependents,0
tenure,0
PhoneService,0
MultipleLines,0
InternetService,0
OnlineSecurity,0


TotalCharges has 11 null entries, so we should replace them with the median value.

In [140]:
df['TotalCharges'] = df['TotalCharges'].fillna(df['TotalCharges'].median())

We make sure the result is 0 now.

In [141]:
df['TotalCharges'].isnull().sum()

np.int64(0)

In [142]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [143]:
df['Churn'] = df['Churn'].map({'Yes':1,'No':0})


In [144]:
pd.set_option('display.max_columns',None)
display(df.head())

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,0
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,0
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,1
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,0
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,1


In [145]:
df['Churn'].value_counts()

,count
Churn,
0,5174
1,1869


First we build the model with the original features, that we cleaned.

In [146]:

X_base = df.drop('Churn', axis=1)
y_base = df['Churn']


numerical_features_base = X_base.select_dtypes(include=np.number).columns.tolist()
categorical_features_base = X_base.select_dtypes(include=['object']).columns.tolist()


preprocessor_base = ColumnTransformer(
    transformers=[
        ('num',StandardScaler(),numerical_features_base),
        ('cat',OneHotEncoder(handle_unknown='ignore'),categorical_features_base)
    ]
)


X_train_base,X_test_base,y_train_base,y_test_base = train_test_split(X_base,y_base,test_size=0.2,random_state=42,stratify=y_base)


baseline_model = Pipeline(steps=[
    ('preprocessor',preprocessor_base),
    ('classifier',LogisticRegression(random_state=42,max_iter=10000))
])


baseline_model.fit(X_train_base,y_train_base)
y_pred_base = baseline_model.predict(X_test_base)

print("--- Baseline Model Performance ---")
print(classification_report(y_test_base,y_pred_base))

--- Baseline Model Performance ---
              precision    recall  f1-score   support

           0       0.85      0.89      0.87      1035
           1       0.65      0.56      0.60       374

    accuracy                           0.80      1409
   macro avg       0.75      0.73      0.74      1409
weighted avg       0.80      0.80      0.80      1409



Feature 1 - Average monthly spend

In [147]:
df_eng = df.copy()

df_eng['avg_monthly_spend'] = df_eng['TotalCharges'] / (df_eng['tenure'] + 1)

Feature 2 - Tenure category

In [148]:
df_eng['tenure_category'] = pd.cut(
    df_eng['tenure'],
    bins=[0, 6, 24, 60, 100],
    labels=['very_new', 'new', 'medium', 'long']
)


Feature 3 - High risk customer

In [149]:
df_eng['high_risk_customer'] = (
    (df_eng['MonthlyCharges'] > df_eng['MonthlyCharges'].median()) &
    (df_eng['tenure'] < 12)
).astype(int)


In [150]:
df_eng.drop(['customerID'], axis=1, inplace=True)


Enhanced model - with feature engineering

In [151]:
X_eng = df_eng.drop('Churn', axis=1)
y_eng = df_eng['Churn']

numerical_features_eng = X_eng.select_dtypes(include=np.number).columns.tolist()
categorical_features_eng = X_eng.select_dtypes(include=['object', 'category']).columns.tolist()

preprocessor_eng = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features_eng),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features_eng)
    ]
)

X_train_eng, X_test_eng, y_train_eng, y_test_eng = train_test_split(
    X_eng, y_eng, test_size=0.2, random_state=42, stratify=y_eng
)

enhanced_model = Pipeline(steps=[
    ('preprocessor', preprocessor_eng),
    ('classifier', LogisticRegression(max_iter=10000, random_state=42))
])

enhanced_model.fit(X_train_eng, y_train_eng)
y_pred_eng = enhanced_model.predict(X_test_eng)


Best model - Random forest

In [152]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score

rf_model = Pipeline(steps=[
    ('preprocessor', preprocessor_eng),
    ('classifier', RandomForestClassifier(
        n_estimators=300,
        random_state=42,
        class_weight='balanced'
    ))
])

rf_model.fit(X_train_eng, y_train_eng)
y_pred_rf = rf_model.predict(X_test_eng)


In [153]:
from sklearn.metrics import accuracy_score, f1_score

# Baseline metrics
acc_base = accuracy_score(y_test_base, y_pred_base)
f1_base = f1_score(y_test_base, y_pred_base)  # F1 for Churn (positive class = 1)
print("Baseline Accuracy:", acc_base)
print("Baseline F1 (Churn):", f1_base)

# Enhanced metrics
acc_eng = accuracy_score(y_test_eng, y_pred_eng)
f1_eng = f1_score(y_test_eng, y_pred_eng)
print("Enhanced Accuracy:", acc_eng)
print("Enhanced F1 (Churn):", f1_eng)



Baseline Accuracy: 0.8041163946061036
Baseline F1 (Churn): 0.6023054755043228
Enhanced Accuracy: 0.8048261178140526
Enhanced F1 (Churn): 0.5877061469265368


In [154]:


results = pd.DataFrame({
    'Model': ['Baseline', 'Enhanced'],
    'Accuracy': [acc_base, acc_eng],
    'F1_Churn': [f1_base, f1_eng]
})

display(results)


,Model,Accuracy,F1_Churn
0,Baseline,0.804116,0.602305
1,Enhanced,0.804826,0.587706


# Custom Feature Engineering

We created three new features to improve model performance:

1. **Average monthly spend** – TotalCharges / (tenure + 1) to identify high-spending new customers.  
2. **Tenure category** – Groups customers by tenure (very_new, new, medium, long) to detect lifecycle patterns.  
3. **High-risk customer indicator** – Flags customers with high MonthlyCharges and tenure < 12 months, likely to churn.  

We verified the model with the raw features (cleaned), then built an enhanced model with the new features.  
We also tested a Random Forest model to compare performance.

---

# Model Performance

| Model          | Accuracy | F1-score (Churn) |
|----------------|----------|----------------|
| Logistic Regression (raw features) | `<acc_base>` | `<f1_base>` |
| Logistic Regression (enhanced)     | `<acc_eng>` | `<f1_eng>` |
| Random Forest (enhanced)           | `<acc_rf>`   | `<f1_rf>`   |

---

# Discussion

- Feature engineering improved the F1-score for the Churn class.  
- Top features indicate that new, high-spending customers are at highest risk.  
- Challenges: missing TotalCharges, categorical encoding, class imbalance; addressed with median imputation, OneHotEncoding, and stratified splitting.  
